In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import gbd_mapping as gbd
from db_queries import get_outputs, get_ids
from get_draws.api import get_draws
from vivarium import Artifact

In [36]:
!whoami
!date

ndbs
Tue Feb  4 14:52:37 PST 2020


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import bep_summarizer

In [3]:
artifact_dir = '/ihme/costeffectiveness/artifacts/vivarium_gates_bep'
results_dir = '/share/costeffectiveness/results/vivarium_gates_bep/base_model'

In [6]:
locations_rundates = {
    'Mali': '2020_02_03_21_34_18',
    'Tanzania': '2020_02_03_21_34_44'
}

In [7]:
all_output = bep_summarizer.load_by_location_and_rundate(results_dir, locations_rundates)
bep_summarizer.print_location_output_shapes(locations_rundates, all_output)

Mali (100, 1384)
Tanzania (100, 1384)


In [8]:
all_output.head()

,,total_population_untracked,total_population_tracked,total_population,diarrheal_diseases_prevalent_cases_at_sim_end,susceptible_to_diarrheal_diseases_event_count,diarrheal_diseases_event_count,measles_prevalent_cases_at_sim_end,susceptible_to_measles_event_count,measles_event_count,recovered_from_measles_event_count,...,gestational_age_mean_among_female,gestational_age_sd_among_female,gestational_age_proportion_below_37w_among_female,birth_weight_mean_among_male,birth_weight_sd_among_male,birth_weight_proportion_below_2500g_among_male,gestational_age_mean_among_male,gestational_age_sd_among_male,gestational_age_proportion_below_37w_among_male,location
input_draw_number,random_seed,,,,,,,,,,,,,,,,,,,,,
357.0,12.0,0.0,10000.0,10000.0,217.0,40972.0,41140.0,107.0,0.0,115.0,111.0,...,38.570552,2.560872,0.144033,2919.088728,658.474258,0.169455,38.632262,2.532454,0.138521,Mali
394.0,70.0,0.0,10000.0,10000.0,186.0,36064.0,36197.0,1017.0,0.0,1067.0,1036.0,...,38.592473,2.547003,0.143675,2906.743706,661.576948,0.177599,38.587616,2.424879,0.138829,Mali
946.0,97.0,0.0,10000.0,10000.0,222.0,38041.0,38177.0,231.0,0.0,239.0,233.0,...,38.572531,2.733318,0.136722,2902.339704,665.288987,0.187066,38.619674,2.538196,0.149614,Mali
602.0,97.0,0.0,10000.0,10000.0,232.0,36664.0,36823.0,516.0,0.0,548.0,522.0,...,38.540375,2.837465,0.149460,2923.239931,651.674749,0.163883,38.560210,2.593945,0.146394,Mali
629.0,36.0,0.0,10000.0,10000.0,181.0,31625.0,31718.0,556.0,0.0,577.0,563.0,...,38.494106,2.875527,0.146860,2910.314342,658.179451,0.179808,38.657477,2.460039,0.125611,Mali


In [10]:
output = bep_summarizer.BEPOutputSummarizer(all_output)
output.print_column_report()

Number of data columns in output: 1384
Total number of columns captured in categories: 1362

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 1, 'intervention': 0, 'run_time': 0, 'diseases_at_end': 5, 'disease_event_count': 311, 'population': 5, 'person_time': 420, 'treated_days': 0, 'mortality': 210, 'total_daly': 2, 'yld': 180, 'yll': 210, 'categorical_risk': 16, 'graded_sequela': 0} 

Missing (22 data column(s) not captured in a category):
 {'gestational_age_mean_among_male', 'stunting_z_score_mean_at_six_months_among_female', 'birth_weight_proportion_below_2500g_among_female', 'gestational_age_sd_among_female', 'gestational_age_sd_among_male', 'wasting_z_score_mean_at_six_months_among_male', 'gestational_age_proportion_below_37w_among_male', 'stunting_z_score_sd_at_six_months_among_male', 'neonatal_disorders_prevalent_count_at_birth_among_female', 'birth_weight_mean_among_female', 'stunting_z_score_sd_at_six_months_among_female', 'birth_weight_sd

### Columns for wasting are exposure in each category at 6 months, WLZ mean and st. dev at 6 months

In [25]:
output.data.filter(regex='wasting').shape

(200, 12)

In [22]:
output.data.filter(regex='wasting').columns

Index(['wasting_cat4_exposed_at_six_months_among_female',
       'wasting_cat3_exposed_at_six_months_among_female',
       'wasting_cat2_exposed_at_six_months_among_female',
       'wasting_cat1_exposed_at_six_months_among_female',
       'wasting_cat4_exposed_at_six_months_among_male',
       'wasting_cat3_exposed_at_six_months_among_male',
       'wasting_cat2_exposed_at_six_months_among_male',
       'wasting_cat1_exposed_at_six_months_among_male',
       'wasting_z_score_mean_at_six_months_among_female',
       'wasting_z_score_sd_at_six_months_among_female',
       'wasting_z_score_mean_at_six_months_among_male',
       'wasting_z_score_sd_at_six_months_among_male'],
      dtype='object')

### 2 "prevalent count" columns: neonatal disorders at birth among male/female

In [16]:
output.data.filter(regex='prevalent_count').head()

,,neonatal_disorders_prevalent_count_at_birth_among_male,neonatal_disorders_prevalent_count_at_birth_among_female
input_draw_number,random_seed,,
357.0,12.0,880.0,741.0
394.0,70.0,926.0,824.0
946.0,97.0,969.0,834.0
602.0,97.0,919.0,869.0
629.0,36.0,775.0,789.0


### 4 "proportion" columns: proportions of LBW and preterm babies by sex

In [24]:
output.data.filter(regex='proportion').columns

Index(['birth_weight_proportion_below_2500g_among_female',
       'gestational_age_proportion_below_37w_among_female',
       'birth_weight_proportion_below_2500g_among_male',
       'gestational_age_proportion_below_37w_among_male'],
      dtype='object')

### Means and standard deviations exist for LAZ/WLZ at 6 months, birth weight, and gestational age

In [26]:
output.data.filter(regex='_mean_').shape

(200, 8)

In [23]:
output.data.filter(regex='_mean_').columns

Index(['wasting_z_score_mean_at_six_months_among_female',
       'stunting_z_score_mean_at_six_months_among_female',
       'wasting_z_score_mean_at_six_months_among_male',
       'stunting_z_score_mean_at_six_months_among_male',
       'birth_weight_mean_among_female', 'gestational_age_mean_among_female',
       'birth_weight_mean_among_male', 'gestational_age_mean_among_male'],
      dtype='object')

In [27]:
output.data.filter(regex='_sd_').shape

(200, 8)

In [21]:
output.data.filter(regex='_sd_').columns

Index(['wasting_z_score_sd_at_six_months_among_female',
       'stunting_z_score_sd_at_six_months_among_female',
       'wasting_z_score_sd_at_six_months_among_male',
       'stunting_z_score_sd_at_six_months_among_male',
       'birth_weight_sd_among_female', 'gestational_age_sd_among_female',
       'birth_weight_sd_among_male', 'gestational_age_sd_among_male'],
      dtype='object')

### Measures at 6 months are stunting/wasting exposure by category, and LAZ/WLZ means and std. devs

In [18]:
output.data.filter(regex='at_six_months').head()

,,wasting_cat4_exposed_at_six_months_among_female,wasting_cat3_exposed_at_six_months_among_female,wasting_cat2_exposed_at_six_months_among_female,wasting_cat1_exposed_at_six_months_among_female,stunting_cat4_exposed_at_six_months_among_female,stunting_cat3_exposed_at_six_months_among_female,stunting_cat2_exposed_at_six_months_among_female,stunting_cat1_exposed_at_six_months_among_female,wasting_cat4_exposed_at_six_months_among_male,wasting_cat3_exposed_at_six_months_among_male,...,stunting_cat2_exposed_at_six_months_among_male,stunting_cat1_exposed_at_six_months_among_male,wasting_z_score_mean_at_six_months_among_female,wasting_z_score_sd_at_six_months_among_female,stunting_z_score_mean_at_six_months_among_female,stunting_z_score_sd_at_six_months_among_female,wasting_z_score_mean_at_six_months_among_male,wasting_z_score_sd_at_six_months_among_male,stunting_z_score_mean_at_six_months_among_male,stunting_z_score_sd_at_six_months_among_male
input_draw_number,random_seed,,,,,,,,,,,,,,,,,,,,,
357.0,12.0,6569.0,1916.0,848.0,315.0,5602.0,2909.0,902.0,235.0,6569.0,1916.0,...,902.0,235.0,9.732495,1.228998,9.732495,1.228998,9.411515,1.457413,9.411515,1.457413
394.0,70.0,6394.0,2198.0,768.0,209.0,3345.0,2504.0,3139.0,581.0,6394.0,2198.0,...,3139.0,581.0,9.805281,1.166262,9.805281,1.166262,9.153314,1.118113,9.153314,1.118113
946.0,97.0,6854.0,1722.0,694.0,346.0,3116.0,3070.0,2190.0,1240.0,6854.0,1722.0,...,2190.0,1240.0,9.848484,1.244168,9.848484,1.244168,9.666802,1.686597,9.666802,1.686597
602.0,97.0,6707.0,1854.0,755.0,278.0,4740.0,3625.0,1077.0,152.0,6707.0,1854.0,...,1077.0,152.0,9.898247,1.258865,9.898247,1.258865,9.431890,1.433573,9.431890,1.433573
629.0,36.0,6609.0,1812.0,848.0,353.0,5846.0,2465.0,982.0,329.0,6609.0,1812.0,...,982.0,329.0,9.806888,1.458078,9.806888,1.458078,9.604427,1.562327,9.604427,1.562327


In [19]:
output.data.filter(regex='at_six_months').columns

Index(['wasting_cat4_exposed_at_six_months_among_female',
       'wasting_cat3_exposed_at_six_months_among_female',
       'wasting_cat2_exposed_at_six_months_among_female',
       'wasting_cat1_exposed_at_six_months_among_female',
       'stunting_cat4_exposed_at_six_months_among_female',
       'stunting_cat3_exposed_at_six_months_among_female',
       'stunting_cat2_exposed_at_six_months_among_female',
       'stunting_cat1_exposed_at_six_months_among_female',
       'wasting_cat4_exposed_at_six_months_among_male',
       'wasting_cat3_exposed_at_six_months_among_male',
       'wasting_cat2_exposed_at_six_months_among_male',
       'wasting_cat1_exposed_at_six_months_among_male',
       'stunting_cat4_exposed_at_six_months_among_male',
       'stunting_cat3_exposed_at_six_months_among_male',
       'stunting_cat2_exposed_at_six_months_among_male',
       'stunting_cat1_exposed_at_six_months_among_male',
       'wasting_z_score_mean_at_six_months_among_female',
       'wasting_z_scor

### 16 risk exposure columns: For stunting/wasting categories at 6 months

In [28]:
len(output.columns('categorical_risk'))

16

In [20]:
output.columns('categorical_risk')

['wasting_cat4_exposed_at_six_months_among_female',
 'wasting_cat3_exposed_at_six_months_among_female',
 'wasting_cat2_exposed_at_six_months_among_female',
 'wasting_cat1_exposed_at_six_months_among_female',
 'stunting_cat4_exposed_at_six_months_among_female',
 'stunting_cat3_exposed_at_six_months_among_female',
 'stunting_cat2_exposed_at_six_months_among_female',
 'stunting_cat1_exposed_at_six_months_among_female',
 'wasting_cat4_exposed_at_six_months_among_male',
 'wasting_cat3_exposed_at_six_months_among_male',
 'wasting_cat2_exposed_at_six_months_among_male',
 'wasting_cat1_exposed_at_six_months_among_male',
 'stunting_cat4_exposed_at_six_months_among_male',
 'stunting_cat3_exposed_at_six_months_among_male',
 'stunting_cat2_exposed_at_six_months_among_male',
 'stunting_cat1_exposed_at_six_months_among_male']

### Hmm, is "prevalent cases at sim end" cumulative over the whole sim?

In [29]:
output.columns('diseases_at_end')

['diarrheal_diseases_prevalent_cases_at_sim_end',
 'measles_prevalent_cases_at_sim_end',
 'lower_respiratory_infections_prevalent_cases_at_sim_end',
 'meningitis_prevalent_cases_at_sim_end',
 'neonatal_disorders_prevalent_cases_at_sim_end']

In [31]:
output.data['neonatal_disorders_prevalent_cases_at_sim_end'].head(10)

input_draw_number  random_seed
357.0              12.0           1398.0
394.0              70.0           1490.0
946.0              97.0           1570.0
602.0              97.0           1541.0
629.0              36.0           1349.0
680.0              7.0            1400.0
232.0              49.0           1381.0
394.0              49.0           1453.0
29.0               44.0           1514.0
232.0              36.0           1418.0
Name: neonatal_disorders_prevalent_cases_at_sim_end, dtype: float64

### What are the 5 population columns?

In [32]:
output.columns('population')

['total_population_untracked',
 'total_population_tracked',
 'total_population',
 'total_population_living',
 'total_population_dead']

### Investigate what the person-time columns look like

In [33]:
output.columns('person_time')[:20]

['person_time_in_2020_among_male_in_age_group_early_neonatal',
 'person_time_in_2020_among_female_in_age_group_early_neonatal',
 'person_time_in_2020_among_male_in_age_group_late_neonatal',
 'person_time_in_2020_among_female_in_age_group_late_neonatal',
 'person_time_in_2020_among_male_in_age_group_1mo_to_6mo',
 'person_time_in_2020_among_female_in_age_group_1mo_to_6mo',
 'person_time_in_2020_among_male_in_age_group_6mo_to_1',
 'person_time_in_2020_among_female_in_age_group_6mo_to_1',
 'person_time_in_2020_among_male_in_age_group_1_to_4',
 'person_time_in_2020_among_female_in_age_group_1_to_4',
 'person_time_in_2021_among_male_in_age_group_early_neonatal',
 'person_time_in_2021_among_female_in_age_group_early_neonatal',
 'person_time_in_2021_among_male_in_age_group_late_neonatal',
 'person_time_in_2021_among_female_in_age_group_late_neonatal',
 'person_time_in_2021_among_male_in_age_group_1mo_to_6mo',
 'person_time_in_2021_among_female_in_age_group_1mo_to_6mo',
 'person_time_in_2021_am

In [34]:
output.columns('person_time')[-20:]

['susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_male_in_age_group_early_neonatal',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_female_in_age_group_early_neonatal',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_male_in_age_group_late_neonatal',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_female_in_age_group_late_neonatal',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_male_in_age_group_1mo_to_6mo',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_female_in_age_group_1mo_to_6mo',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_male_in_age_group_6mo_to_1',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_female_in_age_group_6mo_to_1',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_among_male_in_age_group_1_to_4',
 'susceptible_to_protein_energy_malnutrition_person_time_in_2022_amo

In [35]:
output.columns('person_time')[150:170]

['susceptible_to_measles_person_time_in_2022_among_male_in_age_group_early_neonatal',
 'susceptible_to_measles_person_time_in_2022_among_female_in_age_group_early_neonatal',
 'susceptible_to_measles_person_time_in_2022_among_male_in_age_group_late_neonatal',
 'susceptible_to_measles_person_time_in_2022_among_female_in_age_group_late_neonatal',
 'susceptible_to_measles_person_time_in_2022_among_male_in_age_group_1mo_to_6mo',
 'susceptible_to_measles_person_time_in_2022_among_female_in_age_group_1mo_to_6mo',
 'susceptible_to_measles_person_time_in_2022_among_male_in_age_group_6mo_to_1',
 'susceptible_to_measles_person_time_in_2022_among_female_in_age_group_6mo_to_1',
 'susceptible_to_measles_person_time_in_2022_among_male_in_age_group_1_to_4',
 'susceptible_to_measles_person_time_in_2022_among_female_in_age_group_1_to_4',
 'measles_person_time_in_2022_among_male_in_age_group_early_neonatal',
 'measles_person_time_in_2022_among_female_in_age_group_early_neonatal',
 'measles_person_time_in